In [4]:
"""
Resumo do Código:

Este código lê um arquivo .parquet contendo dados mensais sobre a composição do IBOVESPA, um índice de ações brasileiro. O objetivo é determinar quais ativos são elegíveis para inclusão em uma carteira de investimento, considerando sua presença contínua no IBOVESPA nos últimos 24 meses.

Principais Etapas:
1. Leitura dos dados do arquivo .parquet.
2. Tratamento de empresas que mudaram de ticker, substituindo o ticker antigo pelo novo.
3. Filtragem dos ativos que estiveram no IBOVESPA por mais de 24 meses.
4. Criação de um DataFrame que representa todas as combinações possíveis de datas e ativos.
5. Contagem de meses consecutivos de participação de cada empresa no IBOVESPA.
6. Determinação da elegibilidade dos ativos com base na presença contínua nos últimos 24 meses.
7. Criação de um DataFrame final que indica a elegibilidade de cada ativo em cada data.
8. Salvamento do resultado em um arquivo .parquet chamado 'IBOV_Elegivel.parquet'.

"""

"\nResumo do Código:\n\nEste código lê um arquivo .parquet contendo dados mensais sobre a composição do IBOVESPA, um índice de ações brasileiro. O objetivo é determinar quais ativos são elegíveis para inclusão em uma carteira de investimento, considerando sua presença contínua no IBOVESPA nos últimos 24 meses.\n\nPrincipais Etapas:\n1. Leitura dos dados do arquivo .parquet.\n2. Tratamento de empresas que mudaram de ticker, substituindo o ticker antigo pelo novo.\n3. Filtragem dos ativos que estiveram no IBOVESPA por mais de 24 meses.\n4. Criação de um DataFrame que representa todas as combinações possíveis de datas e ativos.\n5. Contagem de meses consecutivos de participação de cada empresa no IBOVESPA.\n6. Determinação da elegibilidade dos ativos com base na presença contínua nos últimos 24 meses.\n7. Criação de um DataFrame final que indica a elegibilidade de cada ativo em cada data.\n8. Salvamento do resultado em um arquivo .parquet chamado 'IBOV_Elegivel.parquet'.\n\n"

In [1]:
# Importação das bibliotecas necessárias
import pandas as pd
from datetime import datetime
import warnings
import os
warnings.filterwarnings("ignore")

In [2]:
# Define o caminho do arquivo .parquet que contém os dados mensais do IBOVESPA
endereco_IBOV = os.path.join("..","..","..","dataset", "BR", "ACOES" ,"ibov_comp.parquet")

# Lê os dados do arquivo .parquet e os armazena no DataFrame 'IBOV'
IBOV = pd.read_parquet(endereco_IBOV)


#### Renomeando os Tickers das que mudaram de nome

In [3]:
# Obtém a lista de todos os tickers presentes no DataFrame 'IBOV'
todos_ticker_IBOV = IBOV['ticker'].unique()
todos_ticker_IBOV.sort()

# Agrupa os dados pelo ticker e conta o número de ocorrências de cada ticker
IBOV_Grupo = IBOV.groupby("ticker").count()

# Filtra os tickers que estiveram presentes no IBOVESPA por mais de 24 meses
ticker_24 = IBOV_Grupo[IBOV_Grupo['date'] > 24].index

# Filtra o DataFrame original para manter apenas os tickers elegíveis
IBOV_new = IBOV[IBOV['ticker'].isin(ticker_24)]
# Insere uma coluna com o número de dias seguidos que a empresa ficou no IBOVESPA
IBOV_new.insert(len(IBOV_new.columns), "Datas_Seguidas", 0)
# Converte a coluna de datas para o formato datetime
IBOV_new.loc[:,'date'] = pd.to_datetime(IBOV_new['date'], format='%d/%m/%Y')
# Obtém a lista de todas as datas presentes no DataFrame 'IBOV'
todas_datas_IBOV = IBOV_new['date'].unique()
todas_datas_IBOV = sorted(todas_datas_IBOV)
# Converte a coluna 'participacao' de string (com vírgulas) para float
IBOV_new.loc[:,'participacao'] = IBOV_new['participacao'].str.replace(',', '.').astype(float)
# Define a data como índice do DataFrame 'IBOV_new'
IBOV_new.set_index(['date'], inplace=True)


# Crie todas as combinações possíveis de datas e tickers
combinacoes = [(data, ticker) for data in todas_datas_IBOV for ticker in todos_ticker_IBOV]

# Crie um DataFrame com as combinações
df_combinacoes = pd.DataFrame(combinacoes, columns=['date', 'ticker'])

# Una os DataFrames, preenchendo valores ausentes com 0
df_resultante = df_combinacoes.merge(IBOV_new, on=['date', 'ticker'],  how='left').fillna(0)

# Defina o índice como as datas
df_resultante = df_resultante.set_index('date')
IBOV_new = df_resultante.copy()
IBOV_new.insert(len(IBOV_new.columns), "Elegivel_Investimento", False)

# Crie todas as combinações possíveis de datas e tickers
combinacoes = [(data, ticker) for data in todas_datas_IBOV for ticker in todos_ticker_IBOV]

# Crie um DataFrame com as combinações
df_combinacoes = pd.DataFrame(combinacoes, columns=['date', 'ticker'])

# Una os DataFrames, preenchendo valores ausentes com 0
df_resultante = df_combinacoes.merge(IBOV_new, on=['date', 'ticker'],  how='left').fillna(0)



### Contabilizar quantos meses seguidos a empresa ficou no Ibovespa até o mês

In [8]:
IBOV_new[IBOV_new['ticker'] == 'ITUB4']

,ticker,participacao,Datas_Seguidas,Elegivel_Investimento
date,,,,
2010-01-04,ITUB4,8.925,1,False
2010-02-01,ITUB4,8.628,2,False
2010-03-01,ITUB4,8.519,3,False
2010-04-01,ITUB4,8.686,4,False
2010-05-03,ITUB4,8.849,5,False
...,...,...,...,...
2023-05-02,ITUB4,6.388,161,False
2023-06-01,ITUB4,6.276,162,False
2023-07-03,ITUB4,6.285,163,False


In [4]:
# Contabilizar quantos meses seguidos a empresa ficou no Ibovespa até o mês
IBOV_new["Datas_Seguidas"] = IBOV_new.apply(lambda row: 1 if row["participacao"] > 0 else 0, axis=1)
for ticker in ticker_24:
    # Condição para filtrar os lugares onde deseja fazer a atualização
    condicao = (IBOV_new['ticker'] == ticker)
    for i in range(1,len(todas_datas_IBOV)):
        ## Condição para filtrar os lugares onde deseja fazer a atualização
        # Data Anterior
        condicao2 = (IBOV_new.index == todas_datas_IBOV[i-1])
        # Data Atual
        condicao3 = (IBOV_new.index == todas_datas_IBOV[i])

        try:
            dias_anteriores = IBOV_new.loc[condicao & condicao2, 'Datas_Seguidas']
            participacao = IBOV_new.loc[condicao & condicao3, 'participacao'] + 0
        except:
            print(ticker, todas_datas_IBOV[i],"fgdgs")
            raise

        try:
            if participacao[0] > 0:
                IBOV_new.loc[condicao & condicao3, 'Datas_Seguidas'] = dias_anteriores[0] + 1
            else:
                IBOV_new.loc[condicao & condicao3, 'Datas_Seguidas'] = 0
        except:
            print("Erro no if")
            print(i, ticker, todas_datas_IBOV[i])
            print(participacao)
            raise



In [9]:
# Define a coluna 'Elegivel_Investimento' com base na condição de 24 meses seguidos
IBOV_new["Elegivel_Investimento"] = IBOV_new.apply(lambda row: True if row["Datas_Seguidas"] > 24 else False, axis=1)


## Criar um DataFrame com os tickers elegíveis, com index as datas e colunas os tickers
IBOV_Elegiveis = pd.DataFrame(columns=ticker_24, index=todas_datas_IBOV)
IBOV_Elegiveis = IBOV_Elegiveis.fillna(False)
IBOV_Elegiveis = IBOV_Elegiveis.astype(bool)





In [10]:
IBOV_Elegiveis

ticker,ABEV3,AEDU3,ALLL3,ALPA4,ALSC3,ALSO3,ALUP11,AMBV4,AMIL3,ARTR3,...,VALE3,VALE5,VIIA3,VIVT3,VIVT4,VLID3,VVAR11,VVAR3,WEGE3,YDUQ3
2010-01-04,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2010-02-01,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2010-03-01,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2010-04-01,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2010-05-03,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-02,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2023-06-01,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2023-07-03,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2023-08-01,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
# Preenche o DataFrame 'IBOV_Elegiveis' com os valores apropriados
for data in todas_datas_IBOV:
    for ticker in ticker_24:
        IBOV_Elegiveis.loc[data, ticker] =\
              IBOV_new.loc[(IBOV_new.index == data) & (IBOV_new['ticker'] == ticker), 'Elegivel_Investimento'].values[0]

In [24]:

# Define o caminho para salvar o DataFrame 'IBOV_Elegiveis' em formato .parquet
endereco_IBOV_elegivel = os.path.join("..","..","..","dataset", "BR", "ACOES" ,"IBOV_Elegivel.parquet")

# Salva o DataFrame 'IBOV_Elegiveis' em formato .parquet
IBOV_Elegiveis.to_parquet(endereco_IBOV_elegivel, engine='fastparquet')